# Các thư viện được sử dụng

In [60]:
from sympy import *
import numpy as np

# Nhập vào dữ liệu

In [102]:
t = symbols('t')

# Nhập hàm theo biến t
function = cos(t)*sin(t)

# khoảng tích phân
A = -5
B = 1

# số cấp đa thức xấp xỉ
N = 10

# Một số hàm phụ trợ

## Phân tích đa thức $\prod_{i=0,i\neq k}^n (x-i)$ bằng Horner

### Input:
        Cấp n của đa thức
        Số k bị khuyết
### Ouput:
        Mảng các hệ số của đa thức sau khai triển
### Thuật toán:
        

In [40]:
def factorize_poly(n: int, k: int) -> np.array:
    coefs = np.zeros([n, n+2])
    arr = np.arange(n+1)

    coefs[0, n-1], coefs[0, n] = 1, -1
    for i in range(1, n):
        coefs[i, n-i-1] = 1
        for j in range(n-i, n+1):
            coefs[i, j] = coefs[i-1, j+1] - arr[i+1]*coefs[i-1, j]
    
    n_poly = coefs[n-1]
    res = np.zeros(n+1)
    res[0] = 1
    for i in range(1, n+1):
        res[i] = res[i-1]*k + n_poly[i]
    return res

In [41]:
def Horner(arr: np.array, x: int):
    temp = np.zeros(len(arr)+1)
    temp[0] = arr[0]
    for i in range(1, len(arr)):
        temp[i] = temp[i-1]*x + arr[i]
    temp[-1] = temp[len(arr)-1]*x
    return temp[-1]

# Tìm các hệ số Cotes

## Tính tích phân các đa thức trên tử

In [42]:
def numeCoeffs(n: int) -> np.array:
    vacantMatrix = np.zeros([n+1, n+1])
    res = np.zeros(n+1)
    for i in range(n+1):
        vacantMatrix[i] = factorize_poly(n, i)
        for j in range(n+1):
            vacantMatrix[i][j] = 1/(n+1-j) * vacantMatrix[i][j]
        for k in range(n+1):
            res[k] = Horner(vacantMatrix[k], n)
    return res

## Tính các hệ số dưới mẫu

In [43]:
def denoCoeffs(n: int) -> np.array:
    res = np.ones(n+1)
    for i in range(n+1):
        for j in range(n+1):
            if (j != i):
                res[i] *= (i - j)
    return res

# Khai triển thuật toán

In [15]:
N = 5

A = numeCoeffs(N)
B = denoCoeffs(N)

print(A, B, sep="\n")

CotesCoeffs = np.divide(A, B) / N
print(CotesCoeffs)

[-39.58333333  31.25       -10.41666667  10.41666667 -31.25
  39.58333333]
[-120.   24.  -12.   12.  -24.  120.]
[0.06597222 0.26041667 0.17361111 0.17361111 0.26041667 0.06597222]


In [107]:
def my_integrate(func, a, b, n):
    nume = numeCoeffs(n)
    deno = denoCoeffs(n)
    CotesCoeffs = np.divide(nume, deno) / n

    h = (b-a) / n   # step
    res = 0
    xn = a
    for c in CotesCoeffs:
        res += func.subs(t, xn) * c
        xn  += h
    res *= (b-a)
    return res

In [108]:
integral = my_integrate(t**2+3, A, B, N)
print(integral.round(5))

60.0000000000000


### Kiểm tra